In [1]:
using CUTEst, NLPModels, LinearAlgebra;
nlp = CUTEstModel("DIXMAANB")

println(nlp.meta.nvar)

┌ Info: call set_mastsif() to use the full SIF collection
└ @ CUTEst /Users/daniel/.julia/packages/CUTEst/6yQNi/src/CUTEst.jl:73
┌ Info: using problem repository
│   ENV["MASTSIF"] = /usr/local/opt/mastsif/share/mastsif
└ @ CUTEst /Users/daniel/.julia/packages/CUTEst/6yQNi/src/CUTEst.jl:75


3000


In [48]:
	# # S1 := ∑ α*x[i]^2*(i/n)^E[1] for i in 1:n
	# g[i]    += 2*α*x[i]*(i/n)^E[1]
	# g[i+m]  += 2*α*x[i]*(i/n)^E[1]
	# g[i+2m] += 2*α*x[i]*(i/n)^E[1]

	# # S2 := ∑ Β*x[i]^2*(x[i+1]+x[i+1]^2)^2*(i/n)^E[2] for i in 1:n-1
	# g[i]      += 2*Β*x[i]*(x[i+1]+x[i+1]^2)^2*(i/n)^E[2]
	# g[i+m]    += 2*Β*x[i]*(x[i+1]+x[i+1]^2)^2*(i/n)^E[2]
	# g[i+2m]   += 2*Β*x[i]*(x[i+1]+x[i+1]^2)^2*(i/n)^E[2]
	# g[i+1]    += 2*Β*x[i]^2*(x[i+1]+x[i+1]^2)*(i/n)^E[2]*(1+2x[i+1])
	# g[i+m+1]  += 2*Β*x[i]^2*(x[i+1]+x[i+1]^2)*(i/n)^E[2]*(1+2x[i+1])
	# g[i+2m+1] += 2*Β*x[i]^2*(x[i+1]+x[i+1]^2)*(i/n)^E[2]*(1+2x[i+1])

	# # S3 := ∑ γ*x[i]^2*x[i+m]^4*(i/n)^E[3] for i in 1:Int(2*m)
	# g[i]    += 2*γ*x[i]*x[i+m]^4*(i/n)^E[3]
	# g[i+m]  += 2*γ*x[i]*x[i+m]^4*(i/n)^E[3]
	# g[i+m]  += 4*γ*x[i]^2*x[i+m]^3*(i/n)^E[3]
	# g[i+2m] += 4*γ*x[i]^2*x[i+m]^3*(i/n)^E[3]

	# #S4 := ∑ δ*x[i]*x[i+2m]*(i/n)^E[4] for i in 1:m
	# g[i]    += δ*x[i+2m]*(i/n)^E[4]
	# g[i+2m] += δ*x[i]*(i/n)^E[4] 

function f(x)
	n = lastindex(x)
	m = Int(n/3)
	fx = 1.0
	α = 1.0
	Β = 0.0625
	γ = 0.0625
	δ = 0.0625
	E = [0, 0, 0, 0]
	# S1 := ∑ α*x[i]^2*(i/n)^E[1] for i in 1:n
	for i in 1:n
		fx += α*x[i]^2*(i/n)^E[1]
	end

	# S2 := ∑ Β*x[i]^2*(x[i+1]+x[i+1]^2)^2*(i/n)^E[2] for i in 1:n-1
	for i in 1:n-1
		fx += Β*x[i]^2*(x[i+1]+x[i+1]^2)^2*(i/n)^E[2]
	end
		
	# S3 := ∑ γ*x[i]^2*x[i+m]^4*(i/n)^E[3] for i in 1:Int(2*m)
	for i in 1:Int(2*m)
		fx += γ*x[i]^2*x[i+m]^4*(i/n)^E[3]
	end

	#S4 := ∑ δ*x[i]*x[i+2m]*(i/n)^E[4] for i in 1:m
	for i in 1:m
		fx += δ*x[i]*x[i+2m]*(i/n)^E[4]
	end
    return fx
end

function g!(x, g)
	n = lastindex(x)
	m = Int(n/3)
	fx = 1.0
	α = 1.0
	Β = 0.0625
	γ = 0.0625
	δ = 0.0625
	E = [0, 0, 0, 0]

	# S1 := ∑ α*x[i]^2*(i/n)^E[1] for i in 1:n
	for i in 1:n
		g[i] += 2*α*x[i]*(i/n)^E[1]
	end

	# S2 := ∑ Β*x[i]^2*(x[i+1]+x[i+1]^2)^2*(i/n)^E[2] for i in 1:n-1
	for i in 1:n-1
		g[i]   += 2*Β*x[i]*(x[i+1]+x[i+1]^2)^2*(i/n)^E[2]
		g[i+1] += 2*Β*x[i]^2*(x[i+1]+x[i+1]^2)*(i/n)^E[2]*(1+2x[i+1])
	end
		
	# S3 := ∑ γ*x[i]^2*x[i+m]^4*(i/n)^E[3] for i in 1:Int(2*m)
	for i in 1:Int(2*m)
		g[i]   += 2*γ*x[i]*x[i+m]^4*(i/n)^E[3]
		g[i+m] += 4*γ*x[i]^2*x[i+m]^3*(i/n)^E[3]
	end

	#S4 := ∑ δ*x[i]*x[i+2m]*(i/n)^E[4] for i in 1:m
	for i in 1:m
		g[i]    += δ*x[i+2m]*(i/n)^E[4]
		g[i+2m] += δ*x[i]*(i/n)^E[4]
	end
	return g
end


g! (generic function with 1 method)

In [3]:
x0 = copy(nlp.meta.x0)
g  = zeros(nlp.meta.nvar);

show(x0'[1:10])
println("...")
show(x0'[lastindex(x0)-10:lastindex(x0)]) 

[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]...
[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]

In [49]:
f(x0), obj(nlp, x0)


(47242.0, 47242.0)

In [46]:
n = lastindex(x0)
m = Int(n/3)
print(grad(nlp, x0) - g!(x0, copy(g)))


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [47]:
println("Objective difference is same: ", f(x0) ≈ obj(nlp, x0))
println("Gradient normed-difference is same: ", grad(nlp, x0) ≈ g!(x0, copy(g)) )

Objective difference is same: true
Gradient normed-difference is same: true
